In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

times = []

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.15185248851776123
epoch:  1, loss: 0.14543534815311432
epoch:  2, loss: 0.13415062427520752
epoch:  3, loss: 0.12443262338638306
epoch:  4, loss: 0.11456643790006638
epoch:  5, loss: 0.10508512705564499
epoch:  6, loss: 0.09614015370607376
epoch:  7, loss: 0.08828138560056686
epoch:  8, loss: 0.08242309838533401
epoch:  9, loss: 0.07528004795312881
epoch:  10, loss: 0.06879105418920517
epoch:  11, loss: 0.062445588409900665
epoch:  12, loss: 0.05618146434426308
epoch:  13, loss: 0.04970080778002739
epoch:  14, loss: 0.04362639784812927
epoch:  15, loss: 0.038415614515542984
epoch:  16, loss: 0.0338042788207531
epoch:  17, loss: 0.029594436287879944
epoch:  18, loss: 0.026243355125188828
epoch:  19, loss: 0.02301601693034172
epoch:  20, loss: 0.01996503956615925
epoch:  21, loss: 0.01738516241312027
epoch:  22, loss: 0.015019590966403484
epoch:  23, loss: 0.01302588451653719
epoch:  24, loss: 0.011534327641129494
epoch:  25, loss: 0.010068624280393124
epoch:  26, loss

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9899531341772125
Test metrics:  R2 = 0.9809480085976271


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.2768445611000061
epoch:  1, loss: 0.2647181749343872
epoch:  2, loss: 0.25178372859954834
epoch:  3, loss: 0.24008145928382874
epoch:  4, loss: 0.22773155570030212
epoch:  5, loss: 0.21432165801525116
epoch:  6, loss: 0.19943107664585114
epoch:  7, loss: 0.18868085741996765
epoch:  8, loss: 0.17798803746700287
epoch:  9, loss: 0.16768838465213776
epoch:  10, loss: 0.15738897025585175
epoch:  11, loss: 0.14823564887046814
epoch:  12, loss: 0.1385813057422638
epoch:  13, loss: 0.12955832481384277
epoch:  14, loss: 0.12084385007619858
epoch:  15, loss: 0.1124948188662529
epoch:  16, loss: 0.10427310317754745
epoch:  17, loss: 0.09659654647111893
epoch:  18, loss: 0.08893730491399765
epoch:  19, loss: 0.08171466737985611
epoch:  20, loss: 0.07497870177030563
epoch:  21, loss: 0.06853996217250824
epoch:  22, loss: 0.06250086426734924
epoch:  23, loss: 0.05687445029616356
epoch:  24, loss: 0.05165313556790352
epoch:  25, loss: 0.04731715843081474
epoch:  26, loss: 0.042978

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9702296569429305
Test metrics:  R2 = 0.8933314844817699
